In [ ]:
import pickle
import networkx as nx
from shapely.geometry import LineString
from geopy.distance import great_circle
from shapely.geometry import LineString
# .pickle 파일에서 그래프 불러오기
with open(r'C:/my_code/moduroad/moduroad_app/cashe_data/Wheelchair_graph.pickle', 'rb') as file:
    graph = pickle.load(file)

# 모든 엣지에 대해 반복
for u, v, data in graph.edges(data=True):
    # 'geometry' 속성이 있는지 확인하고, 있다면 길이 계산
    if 'geometry' in data:
        # LineString 객체로부터 길이 계산
        line = data['geometry']
        total_distance = 0
        for i in range(len(line.coords)-1):
            point1 = line.coords[i][::-1]  # 순서를 바꿈
            point2 = line.coords[i+1][::-1] 
            segment_distance = great_circle(point1, point2).meters
            total_distance += segment_distance
        # 계산한 길이를 'length' 속성에 저장
        print("ori",data['length'])
        data['length'] = total_distance
        print("chan",data['length'])
    else:
        print("no")

# 변경된 그래프를 다시 .pickle 파일로 저장 (선택적)
#with open(r'C:/my_code/moduroad/moduroad_app/cashe_data/Wheelchair_graph_updated.pickle', 'wb') as file:
    #pickle.dump(graph, file)


In [ ]:
import pickle

# graph 변수에 저장된 그래프 객체가 있다고 가정합니다.
# 예를 들어, graph = nx.Graph()와 같은 네트워크X 그래프 객체일 수 있습니다.

# 파일 저장 경로 설정
file_path = r'C:/my_code/moduroad/moduroad_app/cashe_data/Wheelchair_graph.pickle'
# 'wb'는 바이너리 쓰기 모드를 의미합니다.
with open(file_path, 'wb') as file:
    pickle.dump(graph, file)

print(f"그래프가 성공적으로 {file_path}에 저장되었습니다.")


In [ ]:
import pickle

# 길이 및 장애물별 가중치 정의
length_weight = 0.164  # 길이 당 가중치
obstacle_weights = {
    'slope': 0.195,
    'stair_steep': 0.268,
    'bollard': 0.069,
    'crosswalk_curb': 0.069,
    'sidewalk_curb': 0.069
}

# 그래프 데이터 로드
with open(r'C:/my_code/moduroad/moduroad_app/cashe_data/Wheelchair_graph.pickle', 'rb') as file:
    graph = pickle.load(file)

# 각 엣지에 대한 총 가중치 계산
for u, v, data in graph.edges(data=True):
    # 엣지의 길이를 기본 가중치로 설정
    total_weight = data.get('length', 0) * length_weight
    
    # 각 장애물에 따른 추가 가중치 계산
    for obstacle, weight in obstacle_weights.items():
        if obstacle in data and data[obstacle] != 0:  # 장애물 정보가 있고, 값이 0이 아닌 경우
            num_obstacles = len(data[obstacle]) if isinstance(data[obstacle], list) else 0
            total_weight += num_obstacles * weight
       
    # 계산된 총 가중치를 엣지의 weight 속성에 저장
    data['e_weight'] = total_weight
    print(data)

# 필요한 경우, 가중치가 업데이트된 그래프를 저장
with open(r'C:/my_code/moduroad/moduroad_app/cashe_data/Wheelchair_graph.pickle', 'wb') as file:
    pickle.dump(graph, file)


In [6]:
weights = 'e_weight'
weight_exists = all(weights in data for _, _, data in graph.edges(data=True))

if weight_exists:
    # 모든 간선에 가중치 속성이 존재하면 A* 알고리즘 실행
    print("경로 찾기 성공:")
else:
    # 가중치 속성이 없는 간선이 있으면 경고 메시지 출력
    print(f"경고: '{weights}' 속성이 모든 간선에 존재하지 않습니다.")

경로 찾기 성공:


In [ ]:
for u, v, data in graph.edges(data=True):
    if data['link_id']==